# Лабораторна робота №3: Бустінг (AdaBoost)

**Мета:** Ознайомлення із бустінгом, дослідження його ефективності для задачі класифікації з різними слабкими класифікаторами на прикладі AdaBoost.

In [19]:
# Імпорт необхідних бібліотек
import pandas as pd
import numpy as np
import time # Для вимірювання часу виконання
# StandardScaler може бути не потрібен, якщо використовуємо тільки DT і LinearSVC на масштабованих даних
# from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score, classification_report
import warnings

# Ігнорувати попередження для чистоти виводу
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning)

### Завантаження даних Fashion MNIST з CSV

In [20]:
train_df = pd.read_csv('data/FashionMNIST/fashion-mnist_train.csv')
test_df = pd.read_csv('data/FashionMNIST/fashion-mnist_test.csv')
print("Датасети Fashion MNIST успішно завантажено з CSV.")

Датасети Fashion MNIST успішно завантажено з CSV.


### Підготовка даних

In [21]:
print("\nПочатковий розмір тренувального датасету:", train_df.shape)
print("Початковий розмір тестового датасету:", test_df.shape)

# Розділення на ознаки (X) та мітки (y)
# Перший стовпець ('label') - це мітка класу, решта - пікселі зображення
X_train_raw = train_df.drop('label', axis=1)
y_train = train_df['label']
X_test_raw = test_df.drop('label', axis=1)
y_test = test_df['label']

print(f"\nРозмір навчальних ознак (X_train): {X_train_raw.shape}")
print(f"Розмір навчальних міток (y_train): {y_train.shape}")
print(f"Розмір тестових ознак (X_test): {X_test_raw.shape}")
print(f"Розмір тестових міток (y_test): {y_test.shape}")


Початковий розмір тренувального датасету: (60000, 785)
Початковий розмір тестового датасету: (10000, 785)

Розмір навчальних ознак (X_train): (60000, 784)
Розмір навчальних міток (y_train): (60000,)
Розмір тестових ознак (X_test): (10000, 784)
Розмір тестових міток (y_test): (10000,)


### Попередня обробка: масштабування даних

In [22]:
# Значення пікселів знаходяться в діапазоні [0, 255]. Масштабуємо їх до [0, 1].
# Це важливо для MLP.
X_train = X_train_raw / 255.0
X_test = X_test_raw / 255.0

print("\nМасштабування даних виконано (значення пікселів поділено на 255.0).")
print("Приклад значень після масштабування:", X_train.iloc[0].min(), "-", X_train.iloc[0].max())


Масштабування даних виконано (значення пікселів поділено на 255.0).
Приклад значень після масштабування: 0.0 - 1.0


### Визначення слабких класифікаторів та ансамблів AdaBoost

#### Налаштування слабких класифікаторів

In [23]:
# Використовуємо стандартні гіперпараметри.
# random_state=42 для відтворюваності.

# DecisionTreeClassifier за замовчуванням не має обмеження глибини,
# що робить його не дуже "слабким". Для AdaBoost часто використовують
# неглибокі дерева (stumps - max_depth=1). Використаємо max_depth=1.
dt_stump = DecisionTreeClassifier(max_depth=1, random_state=42)

# LinearSVC з параметрами за замовчуванням.
# Може видавати ConvergenceWarning, якщо max_iter (за замовчуванням 1000) недостатньо.
# dual=False рекомендується, коли n_samples > n_features, що тут вірно.
# Збільшимо max_iter про всяк випадок.
linear_svc_clf = LinearSVC(random_state=42, dual=False, max_iter=2000) # Додано dual=False та збільшено max_iter

print("Слабкі класифікатори визначено: Decision Tree (stump, max_depth=1), LinearSVC.")

Слабкі класифікатори визначено: Decision Tree (stump, max_depth=1), LinearSVC.


#### Створення ансамблів за допомогою AdaBoostClassifier

In [24]:
# Використовуємо AdaBoostClassifier з параметрами за замовчуванням (n_estimators=50).
# Базовий класифікатор передається через параметр `estimator`.

# AdaBoost з деревом рішень (stump)
# Алгоритм за замовчуванням 'SAMME.R' працює добре з деревами
adaboost_dt = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1, random_state=42), # Передаємо новий екземпляр
    n_estimators=50, # Стандартне значення
    random_state=42
)

# AdaBoost з LinearSVC
# Необхідно використовувати алгоритм 'SAMME', оскільки LinearSVC не має методу predict_proba.
adaboost_linearsvc = AdaBoostClassifier(
    estimator=LinearSVC(random_state=42, dual=False, max_iter=2000), # Передаємо новий екземпляр
    n_estimators=50, # Стандартне значення
    random_state=42,
    algorithm='SAMME' 
)

print("Ансамблі AdaBoost створено для Decision Tree (stump) та LinearSVC (з алгоритмом SAMME).")

Ансамблі AdaBoost створено для Decision Tree (stump) та LinearSVC (з алгоритмом SAMME).


### Навчання та оцінка моделей

#### Створення списку моделей для ітерації

In [25]:
# Зменшення вибірки для швидшого виконання
N_SAMPLES_TRAIN = 5000 
N_SAMPLES_TEST = 1000  
X_train_run = X_train[:N_SAMPLES_TRAIN]
y_train_run = y_train[:N_SAMPLES_TRAIN]
X_test_run = X_test[:N_SAMPLES_TEST]
y_test_run = y_test[:N_SAMPLES_TEST]
print(f"ЗАПУСК НА ЗМЕНШЕНІЙ ВИБІРЦІ: {N_SAMPLES_TRAIN} тренувальних, {N_SAMPLES_TEST} тестових.")


models = {
    "слабкий класифікатор 1 (Decision Tree Stump)": dt_stump,
    "ансамбль AdaBoost (Decision Tree Stump)": adaboost_dt,
    "слабкий класифікатор 2 (LinearSVC)": linear_svc_clf,
    "ансамбль AdaBoost (LinearSVC)": adaboost_linearsvc, # Використовує LinearSVC і SAMME
}

results = {} # Словник для зберігання результатів

ЗАПУСК НА ЗМЕНШЕНІЙ ВИБІРЦІ: 5000 тренувальних, 1000 тестових.


#### Цикл навчання та оцінки

In [26]:
print("\n--- Початок навчання та оцінки моделей ---")
for name, model in models.items():
    print(f"\nОбробка моделі: {name}")
    start_time = time.time()

    # Навчання моделі
    try:
        # Для AdaBoost не потрібен Pipeline з препроцесором, бо масштабування вже зроблено
        model.fit(X_train_run, y_train_run)
        fit_time = time.time() - start_time
        print(f"  Навчання завершено за {fit_time:.2f} сек.")

        # Прогнозування на тестовій вибірці
        predict_start_time = time.time()
        y_pred = model.predict(X_test_run)
        predict_time = time.time() - predict_start_time
        print(f"  Прогнозування завершено за {predict_time:.2f} сек.")

        # Оцінка моделі
        accuracy = accuracy_score(y_test_run, y_pred)
        # Використовуємо weighted average для precision та f1-score
        precision = precision_score(y_test_run, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test_run, y_pred, average='weighted', zero_division=0)

        # Збереження результатів
        # У таблиці лабораторної вказано стовпець "Boosting" - вкажемо тип бустінгу
        boosting_type = "AdaBoost" if isinstance(model, AdaBoostClassifier) else "N/A"

        results[name] = {
            "Boosting": boosting_type,
            "Accuracy (test)": accuracy,
            "Precision (test)": precision,
            "F1 Score (test)": f1,
            "Fit Time (s)": fit_time,
            "Predict Time (s)": predict_time
        }

        print(f"  Результати для {name}:")
        print(f"    Accuracy: {accuracy:.4f}")
        print(f"    Precision (weighted): {precision:.4f}")
        print(f"    F1 Score (weighted): {f1:.4f}")

    except Exception as e:
        error_time = time.time() - start_time
        print(f"  ПОМИЛКА під час обробки моделі {name} ({error_time:.2f} сек.): {e}")
        results[name] = {
            "Boosting": "Error",
            "Accuracy (test)": np.nan,
            "Precision (test)": np.nan,
            "F1 Score (test)": np.nan,
            "Fit Time (s)": error_time,
            "Predict Time (s)": 0
        }

print("\n--- Навчання та оцінка завершено ---")


--- Початок навчання та оцінки моделей ---

Обробка моделі: слабкий класифікатор 1 (Decision Tree Stump)
  Навчання завершено за 0.09 сек.
  Прогнозування завершено за 0.00 сек.
  Результати для слабкий класифікатор 1 (Decision Tree Stump):
    Accuracy: 0.1980
    Precision (weighted): 0.0454
    F1 Score (weighted): 0.0726

Обробка моделі: ансамбль AdaBoost (Decision Tree Stump)
  Навчання завершено за 4.47 сек.
  Прогнозування завершено за 0.02 сек.
  Результати для ансамбль AdaBoost (Decision Tree Stump):
    Accuracy: 0.4650
    Precision (weighted): 0.4799
    F1 Score (weighted): 0.4061

Обробка моделі: слабкий класифікатор 2 (LinearSVC)
  Навчання завершено за 1.86 сек.
  Прогнозування завершено за 0.00 сек.
  Результати для слабкий класифікатор 2 (LinearSVC):
    Accuracy: 0.8070
    Precision (weighted): 0.8070
    F1 Score (weighted): 0.8063

Обробка моделі: ансамбль AdaBoost (LinearSVC)
  Навчання завершено за 38.42 сек.
  Прогнозування завершено за 0.03 сек.
  Результати 

In [28]:
print("\n--- Підсумкова таблиця результатів ---")
results_df = pd.DataFrame.from_dict(results, orient='index')

# Перейменування стовпців для відповідності таблиці з лабораторної
results_df = results_df.rename(columns={
    "Accuracy (test)": "Average Accuracy (test)",
    "Precision (test)": "Average Precision (test)", # В лабораторній це "(validation)", але ми рахували на test
    "F1 Score (test)": "F1 score (test)"
})

# Встановлення порядку стовпців як у лабораторній + додамо час
column_order = [
    "Boosting",
    "Average Accuracy (test)",
    "Average Precision (test)", # Додамо реальний Precision на тесті
    "F1 score (test)",
    "Fit Time (s)",
    "Predict Time (s)"
]

# Додаємо відсутні стовпці, якщо їх немає, і перевпорядковуємо
for col in column_order:
    if col not in results_df.columns:
        results_df[col] = "N/A" # Або інше значення за замовчуванням

results_df = results_df[column_order]

# Вивід у форматі Markdown з форматуванням чисел
print(results_df.to_markdown(floatfmt=".4f"))


--- Підсумкова таблиця результатів ---
|                                              | Boosting   |   Average Accuracy (test) |   Average Precision (test) |   F1 score (test) |   Fit Time (s) |   Predict Time (s) |
|:---------------------------------------------|:-----------|--------------------------:|---------------------------:|------------------:|---------------:|-------------------:|
| слабкий класифікатор 1 (Decision Tree Stump) | N/A        |                    0.1980 |                     0.0454 |            0.0726 |         0.0895 |             0.0027 |
| ансамбль AdaBoost (Decision Tree Stump)      | AdaBoost   |                    0.4650 |                     0.4799 |            0.4061 |         4.4667 |             0.0185 |
| слабкий класифікатор 2 (LinearSVC)           | N/A        |                    0.8070 |                     0.8070 |            0.8063 |         1.8557 |             0.0030 |
| ансамбль AdaBoost (LinearSVC)                | AdaBoost   |              